In [1]:
cd ~/Dropbox/project_4

/Users/nbeshouri/Documents/Projects/Metis/Project 4


In [2]:
from hwmf import utils
model = utils.load_data('long_run_on_raw.pickle')

In [4]:
model.docvecs.most_similar('Luke Skywalker')

[('Jacen Solo', 0.876130223274231),
 ('Palpatine', 0.8713411092758179),
 ('Obi-Wan Kenobi', 0.8687915205955505),
 ('R2-D2', 0.8575220704078674),
 ('Han Solo', 0.8573705554008484),
 ('Chewbacca', 0.8343403339385986),
 ('C-3PO', 0.8338310718536377),
 ('Yoda', 0.8125448822975159),
 ('Lando Calrissian', 0.8032432794570923),
 ('Mace Windu', 0.8005262017250061)]

In [91]:
path = '/Users/nbeshouri/Documents/Projects/Metis/Project 4/hwmf/data/character_bios.pickle'

In [92]:
import joblib
import re
title_to_article = joblib.load(path)

In [135]:
new_version = {}
for key, value in title_to_article.items():
    if 'Category:' not in key:
        new_version[key] = value

In [136]:
len(new_version)

15325

In [142]:
joblib.dump(new_version, '/Users/nbeshouri/Documents/Projects/Metis/Project 4/hwmf/data/character_bios.pickle', compress=3)

['/Users/nbeshouri/Documents/Projects/Metis/Project 4/hwmf/data/character_bios.pickle']

In [93]:
s = pd.Series(title_to_article)

In [94]:
sum(s.str.contains(r'=(?:.*)biography(?:.*)=', re.IGNORECASE)) / len(s)

0.2548021880167918

In [144]:
article = title_to_article['Zuko']

In [146]:
def de_wiki(text):
    patterns_to_remove = [
        r'\<ref.*?\</ref\>', 
        r'{{[^{]+?}}',  # Some of these do contain content, e.g. {{c.|lk=no|1100}} and {{nihongo|'''Prince Zuko'''|祖寇|Zǔ Kòu}}
        r'{{[^{]+?}}',  # HACK: Repeat pattern for nested versions that sometimes appear.
        r'{{[^{]+?}}',
        r'\[\[File:.+?(?=\n)',
        r'={2,5}((See also)|(Notes)|(Sources)|(References))={2,5}.*'
    ]
    
    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text, flags=(re.MULTILINE | re.DOTALL))
    
    patterns_to_sub = [
        (r'\[\[([^|]+?)\]\]', r'\1'),
        (r'\[\[(.+?)\|(.+?)\]\]', r'\2'), 
        (r"'''(.+?)'''", r'\1'),
        (r"''(.+?)''", r"\1"),
        (r"<nowiki>(.+?)</nowiki>", r"\1"), 
        (r'\n+', r'\n\n')
    ]
    
    for pattern, replacement in patterns_to_sub:
        text = re.sub(pattern, replacement, text)
        
    return text.strip()
                   

article_2 = de_wiki(article)

In [168]:
print(de_wiki(article))

is a fictional character in Nickelodeon's animated television series Avatar: The Last Airbender. Created by Michael Dante DiMartino and Bryan Konietzko, the character is voiced by Dante Basco and is portrayed by Dev Patel in M. Night Shyamalan's 2010 film The Last Airbender.

Zuko is the Fire Prince of the Fire Nation and an incredibly powerful Firebender, meaning he has the elemental ability to create and control fire and to redirect lightning through martial arts. He is the eldest child of Fire Lord Ozai and Princess Ursa, the older brother of Princess Azula and older half-brother of Kiyi. Before the events of the series, Zuko is exiled from the Fire Nation by his father and told he must capture the Avatar to restore his honor and right to the throne. Zuko is accompanied and advised in his search by his uncle, Iroh. Over time, Zuko sympathizes with oppressed peoples, and joins the Avatar to restore peace. Zuko has two well known great-grandfathers: on his father's side, Fire Lord Soz

In [171]:
text = de_wiki(article)

def get_paragraphs(text):
    text = re.sub(r'==.+==', '', text)
    paragraphs = [p.strip() for p in text.split('\n')]
    return [p for p in paragraphs if p]


# def get_paragraphs(text, sections_):
    
    

for p in get_paragraphs(text):
    print(p)
    print('-------------')

is a fictional character in Nickelodeon's animated television series Avatar: The Last Airbender. Created by Michael Dante DiMartino and Bryan Konietzko, the character is voiced by Dante Basco and is portrayed by Dev Patel in M. Night Shyamalan's 2010 film The Last Airbender.
-------------
Zuko is the Fire Prince of the Fire Nation and an incredibly powerful Firebender, meaning he has the elemental ability to create and control fire and to redirect lightning through martial arts. He is the eldest child of Fire Lord Ozai and Princess Ursa, the older brother of Princess Azula and older half-brother of Kiyi. Before the events of the series, Zuko is exiled from the Fire Nation by his father and told he must capture the Avatar to restore his honor and right to the throne. Zuko is accompanied and advised in his search by his uncle, Iroh. Over time, Zuko sympathizes with oppressed peoples, and joins the Avatar to restore peace. Zuko has two well known great-grandfathers: on his father's side, 

In [78]:
import spacy
nlp = spacy.load('en_core_web_lg')
# Due to a bug, this language model doesn't contain
# stop words, so we fix that here.
for word in nlp.Defaults.stop_words:
    lex = nlp.vocab[word]
    lex.is_stop = True

In [120]:
from spacy import displacy
def show_parse(sentence):
    parsed = nlp(sentence)
#     for token in parsed:
#         print(token.text, token.dep_)
    
    style = 'dep'
    svg = displacy.render(parsed, style='dep')
    with open('sentence.svg', 'w') as f:
        f.write(svg)

    displacy.render(parsed, style='dep', jupyter=True)
    
    return parsed

doc = nlp(article_2)

In [172]:
show_parse('He did it')

He did it

In [128]:
show_parse("Approximately 100 years before the series' timeline, he taught a young (not yet Professor) Wernstrom, whom Farnsworth regarded as a prized student. After he returned a pop quiz to Wernstrom with a grade of A-minus (for poor penmanship), the two became bitter rivals."

A self-described mad scientist, the Professor is a senile, amoral, deranged, and unpredictable old man.

In [109]:
text = """A self-described mad scientist, the Professor is a senile, amoral, deranged, and unpredictable old man (160 years old as of "A Clone of My Own"), with very thick glasses and a gift for creating doomsday devices and atomic supermen. He has put at least one parallel universe in peril with his inventions and visited dozens more (see The Farnsworth Parabox).

The Professor teaches at Mars University and has worked for Momcorp on several occasions, but spends most of his time inventing ridiculous devices and sending the Planet Express delivery crew on suicide missions. While at Momcorp, he fell in love with the CEO, Mom, only to leave her and Momcorp when she decided to weaponize his "Q.T. McWhiskers" toy. What he is a professor of is never explicitly stated. In the episode Mars University when asked what he is teaching, he responds: "The same thing I teach every semester, the mathematics of quantum neutrino fields. I made up the title so no student would dare take it"; however, this declaration has not precluded the professor from demonstrating mastery of whatever field of science is convenient for a given episode's plot, as shown in Bender's Big Score when he proclaims, "I can wire anything directly into anything! I am the Professor!", proceeding to link Hermes' disembodied head to the ship's computer. Approximately 100 years before the series' timeline, he taught a young (not yet Professor) Wernstrom, whom Farnsworth regarded as a prized student. After he returned a pop quiz to Wernstrom with a grade of A-minus (for poor penmanship), the two became bitter rivals (established in "A Big Piece of Garbage")."""

article_title = 'Professor Farnsworth'
article_title = article_title.lower()

relevant_subjects = []
doc = nlp(text)
for sent in doc.sents:
    for token in sent:
        if token.dep_ == 'nsubj' and token.text.lower() in article_title:
            relevant_subjects.append(token)

* You'll need to be able to get the whole noun phrase and replace these with cannonical values.
* I'd use the parser to replace proper nouns, because it seems to be more accurate. As a post processings step, you replace all other named entities like companies, places, etc.
* For now you'll only use sentences where they're the subject... maybe direct ob.
* "whom, Farnsworth, regarded, as, a, prized, student" shows the relative clause problem... you can see the gap there.

In [127]:
for subject in relevant_subjects:
    print('Head:', subject.head)
    for child in subject.head.children:
        print('  ', child, child.dep_, child.pos_)


Head: is
   scientist nsubj NOUN
   Professor nsubj PROPN
   man attr NOUN
   . punct PUNCT
Head: teaches
   Professor nsubj PROPN
   at prep ADP
   and cc CCONJ
   worked conj VERB
   . punct PUNCT
Head: regarded
   whom dobj NOUN
   Farnsworth nsubj PROPN
   as prep ADP


In [131]:
for subject in relevant_subjects:
    print(list(subject.head.subtree))


[A, self, -, described, mad, scientist, ,, the, Professor, is, a, senile, ,, amoral, ,, deranged, ,, and, unpredictable, old, man, (, 160, years, old, as, of, ", A, Clone, of, My, Own, ", ), ,, with, very, thick, glasses, and, a, gift, for, creating, doomsday, devices, and, atomic, supermen, .]
[The, Professor, teaches, at, Mars, University, and, has, worked, for, Momcorp, on, several, occasions, ,, but, spends, most, of, his, time, inventing, ridiculous, devices, and, sending, the, Planet, Express, delivery, crew, on, suicide, missions, .]
[whom, Farnsworth, regarded, as, a, prized, student]


In [117]:
list(subject.head.children)

[whom, Farnsworth, as]

In [100]:
for ent in doc.ents:
    print('ent:', ent, 'label:', ent.label_)

ent: Bender Bending Rodríguez label: PERSON
ent: Bending Unit label: ORG
ent: 1,729 label: CARDINAL
ent: 2716057 label: CARDINAL
ent: one label: CARDINAL
ent: Futurama label: WORK_OF_ART
ent: Matt Groening label: PERSON
ent: David X. Cohen label: PERSON
ent: John DiMaggio label: PERSON
ent: Futurama label: PERSON
ent: Leela label: PERSON
ent: 

 label: PERSON
ent: Bender label: PERSON
ent: Tijuana label: GPE
ent: Mexico label: GPE
ent: Latin label: NORP
ent: Bender label: PERSON
ent: one label: CARDINAL
ent: Bender label: PERSON
ent: Do Not Kill label: WORK_OF_ART
ent: Philip J. Fry label: PERSON
ent: Hermes Conrad label: PERSON
ent: Hermes label: ORG
ent: Lethal Inspection label: WORK_OF_ART
ent: 

 label: CARDINAL
ent: biography== label: PERSON
ent: 

Bender label: PERSON
ent: 2996 label: DATE
ent: Fábrica Robótica De La Madre label: FAC
ent: Spanish label: NORP
ent: Robot Factory label: ORG
ent: Mom's label: ORG
ent: Friendly Robot Company label: ORG
ent: Tijuana label: GPE
ent: Mex